In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
from rich import print
import warnings
import datetime
from astropy.time import Time
import os

In [2]:
def counts2mag(cps, band):
    scale = 18.82 if band == 'FUV' else 20.08
    with np.errstate(invalid='ignore'):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            mag = -2.5 * np.log10(cps) + scale
    return mag

In [3]:
header_data = pd.read_csv('../ref/mislike_image_header_table.csv')
catalog_filename = '../ref/catalog_nd_daostarfinder.parquet'
catalog_file = pq.ParquetFile(catalog_filename)

In [4]:
flare_gifs = !ls /Users/cm/GFCAT/gfcat/test_gifs/sorted/flare/*
print(len(flare_gifs))

528

In [5]:
def make_target_table(fn_list):
    table = pd.DataFrame()
    for fn_list in fn:
        eclipse = int(fn.split('/')[-1].split('-')[0][1:])
        obj_id = int(fn.split('/')[-1].split('-')[1])
        this_star = pq.read_table(catalog_filename,filters =
                                  [('eclipse','=',eclipse),
                                   ('obj_id','=',obj_id)]).to_pandas()
        table = pd.concat([table,this_star[["eclipse","obj_id","ra","dec",]]])


In [6]:
['marginal','eclipse','trend','flare','unk variable','moving']
print(os.listdir("/Users/cm/GFCAT/gfcat/test_gifs/sorted/"))

[
    'marginal',
    'adjust',
    '.DS_Store',
    'eclipse',
    'trend',
    'moving',
    'adjust2',
    'flares_adjust',
    'artifact',
    'unk variable',
    'flare'
]

In [12]:
flare_gifs = !ls /Users/cm/GFCAT/gfcat/test_gifs/sorted/flare/*
flare_table = pd.DataFrame()
for flare_gif in flare_gifs:
    eclipse = int(flare_gif.split('/')[-1].split('-')[0][1:])
    obj_id = int(flare_gif.split('/')[-1].split('-')[1])
    this_star = pq.read_table(catalog_filename,filters =
                              [('eclipse','=',eclipse),
                               ('obj_id','=',obj_id)]).to_pandas()
    flare_table = pd.concat([flare_table,this_star[["eclipse","obj_id","ra","dec",]]])
flare_table.to_csv('flare_table.csv',index=None)

In [13]:
print(flare_table)

eclipse      obj_id          ra        dec
0      1413  1953301413  311.565113  -4.930670
0      1420  1468701420  315.935391  -7.379244
0      1422  1479501422  343.352654 -39.793072
0      1623  1417401623  259.204219  60.039535
0      1640  1297001640  260.307220  58.102305
..      ...         ...         ...        ...
0     45702  1401745702   25.245244  34.078142
0     45730  1513045730   27.571128  31.047408
0     45856  1995745856  354.573904 -41.518851
0     46287   637246287   35.457369  74.264867
0     46657  4738446657  135.403098 -33.594075

[528 rows x 4 columns]

In [82]:
def make_variable_target_table(gif_fns):
    variable_table = pd.DataFrame()
    for gif_fn in gif_fns:
        eclipse = int(flare_gif.split('/')[-1].split('-')[0][1:])
        obj_id = int(flare_gif.split('/')[-1].split('-')[1])
        this_star = pq.read_table(catalog_filename,filters =
                                  [('eclipse','=',eclipse),
                                   ('obj_id','=',obj_id)]).to_pandas()
        expt = header_data.loc[header_data['ECLIPSE']==eclipse].loc[header_data['BAND']=='NUV']['EXPT_0']
        cps = this_star["aperture_sum_n_12_8"].tolist()[0]/expt.tolist()[0]
        cps_err = np.sqrt(this_star["aperture_sum_n_12_8"].tolist()[0])/expt.tolist()[0]
        mag = counts2mag(cps,'NUV')
        mag_err_1 = mag-counts2mag(cps+cps_err,'NUV')
        mag_err_2 = counts2mag(cps-cps_err,'NUV')-mag # this one is always larger
        this_star['cps'] = cps
        this_star['cps_err'] = cps_err
        this_star['NUVmag'] = mag_err_1
        this_star['NUVmag_err_1'] = mag_err_1
        this_star['NUVmag_err_2'] = mag_err_2
        variable_table = pd.concat([variable_table,
                                 this_star[
                                    ["obj_id","ra","dec","eclipse","NUVmag","NUVmag_err_1","NUVmag_err_2",]]])
    return variable_table


obj_id          ra        dec  eclipse    NUVmag  NUVmag_err_1  \
0   1953301413  311.565113  -4.930670     1413  0.010390      0.010390   
0   1468701420  315.935391  -7.379244     1420  0.021507      0.021507   
0   1479501422  343.352654 -39.793072     1422  0.039535      0.039535   
0   1417401623  259.204219  60.039535     1623  0.038897      0.038897   
0   1297001640  260.307220  58.102305     1640  0.030795      0.030795   
..         ...         ...        ...      ...       ...           ...   
0   1401745702   25.245244  34.078142    45702  0.024567      0.024567   
0   1513045730   27.571128  31.047408    45730  0.016177      0.016177   
0   1995745856  354.573904 -41.518851    45856  0.022697      0.022697   
0    637246287   35.457369  74.264867    46287  0.024948      0.024948   
0   4738446657  135.403098 -33.594075    46657  0.025355      0.025355   

    NUVmag_err_2  
0       0.010491  
0       0.021941  
0       0.041029  
0       0.040342  
0       0.031694  
..           ...  
0       0.025136  
0       0.016422  
0       0.023181  
0       0.025535  
0       0.025961  

[545 rows x 7 columns]

In [ ]:
#ID, ra, dec, epoch_GALEX , [FUVmag , NUVmag, FUVmagerr, NUVmagerr ]

In [76]:
this_star[["obj_id","ra","dec","eclipse","NUVmag","NUVmag_err_1","NUVmag_err_2",]]

,eclipse,obj_id,ra,dec,mag,mag_err_1,mag_err_2,cps,cps_err
0,1640,1297001640,260.30722,58.102305,0.030795,0.030795,0.031694,0.995405,0.028637


In [50]:
expt = header_data.loc[header_data['ECLIPSE']==eclipse].loc[header_data['BAND']=='NUV']['EXPT_0']

In [57]:
cps = this_star["aperture_sum_n_12_8"].tolist()[0]/expt.tolist()[0]

0.9954052112383378

In [56]:
expt.tolist()[0]

1213.79600832833

In [29]:
this_star['mag']

0    20
Name: mag, dtype: int64

In [83]:
    this_star = pq.read_table(catalog_filename,filters =
                              [('eclipse','=',eclipse),
                               ('obj_id','=',obj_id)]).to_pandas()


In [105]:
GPSSECS = 315532800+432000
t = header_data['T0_0'].iloc[0]+GPSSECS
print(t)
dt = datetime.datetime.fromtimestamp(t)

1054985779.995

In [109]:
dt.isoformat().split('.')[0]

'2003-06-07T07:36:19'

In [112]:
Time(dt,format='datetime').decimalyear

2003.4310052002472